In [8]:
import torch
import numpy as np

In [5]:
B = 4
feat_dim = 2048
H = 33
W = 33

features = torch.randn(B, feat_dim, H, W)
print(features.shape)

torch.Size([4, 2048, 33, 33])


In [6]:
labels = torch.randint(0, 5, (B, H ,W))
print(labels.shape)

torch.Size([4, 33, 33])


In [9]:
source_class_prototypes = np.load('pretrained_models/mr2ct_class_center_feas_init.npy')
source_class_prototypes = torch.from_numpy(source_class_prototypes).float().cuda()
print(source_class_prototypes.shape)

torch.Size([5, 2048])


In [10]:
source_class_prototypes

tensor([[5.4051e-02, 2.0090e-02, 1.7574e-02,  ..., 1.5758e-02, 8.6078e-02,
         7.0926e-03],
        [1.3640e-02, 1.0893e-02, 3.4559e-01,  ..., 8.1896e-02, 1.2376e-03,
         9.9656e-03],
        [1.6323e-01, 1.6255e-03, 6.8898e-01,  ..., 6.2445e-02, 7.1961e-05,
         2.0323e-02],
        [2.6903e-02, 2.2649e-04, 1.2833e-01,  ..., 5.9504e-02, 2.1416e-04,
         1.0025e-01],
        [1.6706e-02, 6.8699e-04, 2.7064e-01,  ..., 1.4282e-03, 1.4742e-03,
         1.2156e-02]], device='cuda:0')

In [12]:
import os
import pprint
import random
import warnings
import yaml
import torch
import argparse
import numpy as np
import os.path as osp
from torch.utils import data

from train import train_supervised, train_supervised_etf, train_uda_dap
from data_reader import CTDataset, MRDataset, CTDataset_aug, MRDataset_aug
from model.deeplabv2 import get_deeplab_v2

model = get_deeplab_v2(num_classes=5, multi_level=True)
saved_state_dict = torch.load('pretrained_models/DeepLab_resnet_pretrained_imagenet.pth', weights_only=True, map_location=torch.device("cpu"))
if 'DeepLab_resnet_pretrained' in 'pretrained_models/DeepLab_resnet_pretrained_imagenet.pth':
    new_params = model.state_dict().copy()
    for i in saved_state_dict:
        i_parts = i.split('.')
        if not i_parts[1] == 'layer5':
            new_params['.'.join(i_parts[1:])] = saved_state_dict[i]
    model.load_state_dict(new_params)
else:
    model.load_state_dict(saved_state_dict)

model = torch.compile(model)

In [18]:
transforms = None
img_mean = np.array((104.00698793, 116.66876762, 122.67891434), dtype=np.float32)
train_data = 'data/datalist/train_mr.txt'
train_gt_data = 'data/datalist/train_mr_gt.txt'

def _init_fn(worker_id):
        np.random.seed(0+worker_id)

train_dataset = MRDataset(data_pth=train_data, gt_pth=train_gt_data, img_mean=img_mean, transform=transforms)
train_loader = data.DataLoader(train_dataset,
                                      batch_size=4,
                                      shuffle=True,
                                      pin_memory=True,
                                      worker_init_fn=_init_fn)

In [ ]:
train_loader_